# Suggested Credential Optimization (BMC Discovery)

Replicates the DisMAL `suggested_cred_opt` report by ranking vault credentials
based on scope, ranges/exclusions, type, key/version hints, and recent success/failure.
Outputs `suggested_cred_opt.csv` under `output_<target>`.

> **NOTE:** This can take a while to run.

## Requirements
We use `tideway` (SDK) for credentials, plus `requests`, `pandas`, and `PyYAML`.

In [ ]:
# %pip install -q tideway requests pandas pyyaml

import os, sys, subprocess
from pathlib import Path
from typing import Any, Dict, List, Tuple, Optional
import pandas as pd
import requests
import yaml
from xml.etree import ElementTree as ET


## Select Appliance (optional)
Set `APPLIANCE_NAME` or `APPLIANCE_INDEX` when `config.yaml` lists multiple appliances.

In [ ]:
APPLIANCE_NAME = None
APPLIANCE_INDEX = 0


## Configuration (from config.yaml)
Locates repo root, reads target/token, initialises Tideway appliance, and prepares output folder.

In [ ]:
def _find_repo_root(start: Path) -> Path:
    for p in [start] + list(start.parents):
        if (p / 'config.yaml').exists():
            return p
    return start.parent

repo_root = _find_repo_root(Path.cwd())
cfg = yaml.safe_load(open(repo_root / 'config.yaml', 'r')) or {}
apps = cfg.get('appliances') or []
selected = None
if isinstance(apps, list) and apps:
    if APPLIANCE_NAME:
        selected = next((a for a in apps if a.get('name') == APPLIANCE_NAME), None)
        if selected is None:
            raise ValueError(f"No appliance named '{APPLIANCE_NAME}' in config.yaml")
    else:
        try:
            selected = apps[int(APPLIANCE_INDEX)]
        except Exception:
            selected = apps[0]
target = ((selected or {}).get('target') or cfg.get('target') or '').strip()
if not target:
    raise ValueError('config.yaml missing "target"')
token = (((selected or {}).get('token') or cfg.get('token') or '').strip())
token_file = (selected or {}).get('token_file') or cfg.get('token_file') or cfg.get('f_token')
if not token and token_file:
    tf_path = Path(token_file)
    if not tf_path.is_absolute():
        tf_path = repo_root / tf_path
    token = open(tf_path, 'r').read().strip()
if not token:
    raise ValueError('API token not found in config.yaml (token or token_file)')
API_VERSION = str((selected or {}).get('api_version') or cfg.get('api_version') or 'v1.14')
VERIFY_SSL = bool((selected or {}).get('verify_ssl', cfg.get('verify_ssl', True)))
sanitized = target.replace('.', '_').replace(':', '_').replace('/', '_')
output_dir = repo_root / f'output_{sanitized}'
output_dir.mkdir(parents=True, exist_ok=True)

# Tideway SDK
try:
    import tideway  # type: ignore
except Exception:
    print('Installing tideway via pip...')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'tideway'])
    import tideway  # retry

API_VERSION_NUM = API_VERSION.lstrip('v')
app = tideway.appliance(target, token, api_version=API_VERSION_NUM, ssl_verify=VERIFY_SSL)
twsearch = app.data()
twcreds = app.credentials()

# Requests session for Data API + Outposts
session = requests.Session()
auth_value = token if token.lower().startswith('bearer ') else f'Bearer {token}'
session.headers.update({'Authorization': auth_value, 'Accept': 'application/json'})
session.verify = VERIFY_SSL
BASE_URL = target if ('://' in target) else f'https://{target}'

print('Base Host      :', target)
print('API Version    :', API_VERSION)
print('Verify SSL     :', VERIFY_SSL)
print('Output folder  :', output_dir)


In [ ]:
queries_xml_candidates = [
    repo_root / 'queries' / 'dismal_queries.xml',
    repo_root / 'DisMAL' / 'queries' / 'dismal_queries.xml',
]
for candidate in queries_xml_candidates:
    if candidate.exists():
        queries_xml = candidate
        break
else:
    raise FileNotFoundError('Unable to locate dismal_queries.xml')

def load_query_from_xml(title: str, xml_path: Optional[Path] = None) -> str:
    xml_path = xml_path or queries_xml
    tree = ET.parse(xml_path)
    root = tree.getroot()
    for elem in root.findall('query'):
        if (elem.get('title') or '').strip() == title:
            search_text = (elem.findtext('search') or '').strip()
            if not search_text:
                raise ValueError(f"Query '{title}' missing search text")
            return search_text
    raise ValueError(f"Query '{title}' not found in {xml_path}")

QUERY_OUTPOST_SESSION_TITLE = 'Suggested Cred Opt - Session Results Map'


## Helpers
Unified post_search for TWQL, token parsing helpers, outpost mapping, and weighting rules.

In [ ]:
from urllib.parse import urljoin

def api_url(path: str) -> str:
    base = BASE_URL.rstrip('/') + f'/api/{API_VERSION}/'
    return urljoin(base, path.lstrip('/'))

def post_search(query: str, *, limit: int | None = None, page_size: int = 500):
    url = api_url('data/search')
    headings = None
    results = []
    offset = 0
    fetch_all = (limit == 0)
    while True:
        payload = {'query': query, 'format': 'object'}
        if fetch_all:
            payload['limit'] = page_size
            if offset:
                payload['offset'] = offset
        elif limit is not None:
            payload['limit'] = limit
        r = session.post(url, json=payload)
        if r.status_code >= 400:
            print(f'Error {r.status_code} POST {url}: {r.text[:200]}')
            return {'headings': [], 'results': []}
        data = {}
        try:
            data = r.json()
        except Exception:
            data = {}
        table = None
        if isinstance(data, list):
            for x in data:
                if isinstance(x, dict) and 'headings' in x and 'results' in x:
                    table = x; break
        elif isinstance(data, dict) and 'headings' in data and 'results' in data:
            table = data
        if not table:
            return {'headings': [], 'results': []}
        if headings is None:
            headings = table.get('headings', [])
        rows = table.get('results') or []
        results.extend(rows)
        if not fetch_all or len(rows) < page_size:
            break
        offset += page_size
    return {'headings': (headings or []), 'results': results}

def get_vault_credentials() -> List[Dict[str, Any]]:
    ep = twcreds.get_vault_credentials
    try:
        resp = ep() if callable(ep) else ep
    except Exception:
        resp = ep
    try:
        return resp.json() if hasattr(resp, 'json') else (resp or [])
    except Exception:
        return []

def outpost_url_map() -> Dict[str, str]:
    # Build uuid -> outpost URL using data query + outposts endpoint
    tbl = post_search(load_query_from_xml(QUERY_OUTPOST_SESSION_TITLE), limit=0)
    heads = tbl.get('headings', [])
    rows = tbl.get('results', [])
    col_uuid = heads.index('uuid') if 'uuid' in heads else None
    col_outpost = heads.index('outpost') if 'outpost' in heads else None
    uuid_to_opid = {}
    if col_uuid is not None and col_outpost is not None:
        for r in rows:
            try:
                uuid_to_opid[str(r[col_uuid])] = str(r[col_outpost])
            except Exception:
                pass
    # Get outposts list
    url = api_url('discovery/outposts?deleted=false')
    try:
        r = session.get(url)
        ops = r.json() if r.ok else []
    except Exception:
        ops = []
    id_to_url = {}
    for op in (ops or []):
        if not isinstance(op, dict):
            continue
        op_id = op.get('id') or op.get('outpost') or op.get('outpost_id') or op.get('uuid')
        url = op.get('url')
        if op_id and url:
            id_to_url[str(op_id)] = url
    return {u: id_to_url.get(opid) for u, opid in uuid_to_opid.items()}

def _tokens_from_value(value) -> List[str]:
    if isinstance(value, list):
        tokens = []
        for v in value:
            if isinstance(v, str):
                tokens.extend([x.strip() for x in v.split(',') if x.strip()])
    elif isinstance(value, str):
        tokens = [x.strip() for x in value.split(',') if x.strip()]
    else:
        tokens = []
    return tokens

def compute_weight(cred: Dict[str, Any]) -> int:
    w = 100
    label = cred.get('label')
    # IP range
    ip_tokens = _tokens_from_value(cred.get('ip_range'))
    for t in ip_tokens:
        if t in ('0.0.0.0/0', '::/0', '0.0.0.0/0,::/0'):
            w = 4294967296
        else:
            w += 1
    # Exclusions
    ex_tokens = _tokens_from_value(cred.get('ip_exclusion'))
    for t in ex_tokens:
        if t in ('0.0.0.0/0', '::/0', '0.0.0.0/0,::/0'):
            w = -4294967296
        else:
            w -= 1
    # Types
    for t in (cred.get('types') or []):
        if t in ('aws','openstack','azure','web_basic','google'):
            w += 1
        elif t in ('ssh','powershell'):
            w += 2
        elif t == 'windows':
            w += 3
        elif t in ('vsphere','vcenter'):
            w += 4
        elif t == 'snmp':
            w += 5
        else:
            w += 6
    # Hints
    if cred.get('ssh.key.set'):
        w -= 1
    if cred.get('snmp.version') == 'v3':
        w -= 1
    scopes = cred.get('scopes') or []
    if isinstance(scopes, list) and len(scopes) > 0:
        w -= 1
    return w


## Build suggested order and save
Computes weights, factors in recent success/failure, assigns new indices, and writes CSV.

In [ ]:
creds = get_vault_credentials()
op_map = outpost_url_map()
rows = []
weighted = []
# Precompute base weight per credential
for c in (creds or []):
    try:
        base_w = compute_weight(c)
        weighted.append({'uuid': c.get('uuid'), 'label': c.get('label'), 'current_index': c.get('index'), 'weight': base_w, 'scopes': c.get('scopes') or []})
    except Exception:
        continue

# Adjust weights by recent success/failure via Data API
for w in weighted:
    u = w['uuid']
    if not u:
        continue
    # Successful sessions
    q1 = f"search SessionResult where success and (slave = '{u}' or credential = '{u}') show (credential or slave) as cred_uuid, session_type"
    t1 = post_search(q1, limit=0)
    if t1.get('results'):
        w['weight'] -= len(t1['results'])
    # Successful deviceinfo (without session results)
    q2 = f"search DeviceInfo where method_success and (slave = '{u}' or credential = '{u}') and nodecount(traverse DiscoveryResult:DiscoveryAccessResult:DiscoveryAccess:DiscoveryAccess traverse DiscoveryAccess:Metadata:Detail:SessionResult) = 0 show (last_credential or last_slave) as cred_uuid, access_method as 'session_type'"
    t2 = post_search(q2, limit=0)
    if t2.get('results'):
        w['weight'] -= len(t2['results'])
    # Failures
    q3 = f"search SessionResult where not success and (slave = '{u}' or credential = '{u}') show (credential or slave) as cred_uuid, session_type"
    t3 = post_search(q3, limit=0)
    if t3.get('results'):
        w['weight'] += len(t3['results'])

# Sort and assign new indices
weighted.sort(key=lambda x: x['weight'])
for i, w in enumerate(weighted):
    w['new_index'] = i

# Build output rows
for c in creds or []:
    u = c.get('uuid')
    w = next((x for x in weighted if x['uuid'] == u), None)
    if not w:
        continue
    scope = c.get('scopes') or []
    if isinstance(scope, list):
        scope = ', '.join(scope)
    url = op_map.get(str(u))
    rows.append([c.get('label'), c.get('index'), w.get('weight'), w.get('new_index'), scope, url])

headers = ['Discovery Instance','Credential','Current Index','Weighting','New Index','Scope','Outpost URL']
df = pd.DataFrame(rows, columns=headers[1:])
df.insert(0, 'Discovery Instance', target)
display(df.head(20)) if not df.empty else print('No rows to display')

OUT_CSV = str(output_dir / 'suggested_cred_opt.csv')
df.to_csv(OUT_CSV, index=False)
print(f'Saved to {OUT_CSV} (rows: {len(df)})')
